In [82]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [83]:
df = pd.read_csv("D:/PSG/PSG-Z/SEM 5/20XW58 MACHINE LEARNING LAB/practice/datasets/wbc_original.csv")
del df['id']

In [84]:
df.head()

,clump_thickness,size_uniformity,shape_uniformity,marginal_adhesion,epithelial_size,bare_nucleoli,bland_chromatin,normal_nucleoli,mitoses,class
0,5,1,1,1,2,1,3,1,1,2
1,5,4,4,5,7,10,3,2,1,2
2,3,1,1,1,2,2,3,1,1,2
3,6,8,8,1,3,4,3,7,1,2
4,4,1,1,3,2,1,3,1,1,2


In [85]:
features = df.columns[:-1]

In [86]:
def remove_multicollinearity(features):
    final_features = [x for x in features]
    
    CR = df[features].corr().values.tolist()
    
    for i in range(len(CR)):
        for j in range(i+1,len(CR)):
            if abs(CR[i][j])>0.7 and features[i] in final_features:
                final_features.remove(features[i])
    return final_features

In [87]:
final_features = remove_multicollinearity(features)

In [88]:
df[final_features]

,clump_thickness,marginal_adhesion,epithelial_size,bare_nucleoli,bland_chromatin,normal_nucleoli,mitoses
0,5,1,2,1,3,1,1
1,5,5,7,10,3,2,1
2,3,1,2,2,3,1,1
3,6,1,3,4,3,7,1
4,4,3,2,1,3,1,1
...,...,...,...,...,...,...,...
694,3,1,3,2,1,1,1
695,2,1,2,1,1,1,1
696,5,3,7,3,8,10,2
697,4,4,3,4,10,6,1


In [89]:
print(df['bare_nucleoli'][40])

?


In [90]:
le1 = LabelEncoder()
# le2 = LabelEncoder()

df['class'] = le1.fit_transform(df['class'])

In [91]:
# df['bare_nucleoli'] = le2.fit_transform(df['bare_nucleoli'])

In [92]:
print(df['bare_nucleoli'][40])
# print(sum(df['bare_nucleoli'])/699)

?


In [93]:
df['bare_nucleoli'] = df['bare_nucleoli'].replace('?','1')

In [94]:
X = df[final_features]
Y = df['class']

In [95]:
def makeCPT(x,y,unique_quantities):
    y = list(y)
    
    CPT_YES_FEATURES = {}
    CPT_NO_FEATURES = {}
    
    for k in range(len(x[0])):
        CPT_YES = {}
        CPT_NO = {}
        for j in range(len(unique_quantities)):
            yeslist=[]
            nolist=[]
            for i in range(len(x)):
                if(x[i][k]==j and y[i]==1):
                    yeslist.append(1)
                if(x[i][k]==j and y[i]==1):
                    nolist.append(0)
                    
            CPT_YES[j] = len(yeslist)/y.count(1)
            CPT_NO[j] = len(nolist)/y.count(0)
            
        CPT_YES_FEATURES[final_features[k]] = CPT_YES
        CPT_NO_FEATURES[final_features[k]] = CPT_NO
    
    return (CPT_YES_FEATURES,CPT_NO_FEATURES)

In [96]:
def testNaiveBayes(x_test,y_test,yCPT,nCPT,y_prob):
#     print(x_test)
    Y_posterior = []
    for i in range(len(x_test)):
        num = y_prob
        denom = (1-y_prob)
        for j in range(len(x_test[0])):
            num *= yCPT[final_features[j]][x_test[i][j]]
            denom *= nCPT[final_features[j]][x_test[i][j]]
        denom+=num
        posterior_prob = num/denom
#             print(yCPT[final_features[j]][x_test[i][j]]) 
        Y_posterior.append(posterior_prob)
        
    print(Y_posterior)
    
    for i in range(len(Y_posterior)):
        if (Y_posterior[i]<0.5):
            Y_posterior[i]=0
        else:
            Y_posterior[i]=1
    
    match_count = 0
    for i in range(len(y_test)):
        if (Y_posterior[i]==y_test[i]):
            match_count +=1
    
    acc = match_count/len(y_test)
    
    return acc

In [97]:
def naiveBayesClassifier(x_train,y_train,x_test,y_test):
    no_of_features = len(x_train[0])
#     identifying unique quantities
    unique_quantities = [set()]*7
    
    for j in range(no_of_features):
        for i in range(len(x_train)):
            unique_quantities[j].add(x_train[i][j])

    CPTtup = makeCPT(x_train,y_train,unique_quantities)
    
    yCPT = CPTtup[0]
    nCPT = CPTtup[1]
    print(yCPT)
    print(nCPT)
    y_prob = list(y_train).count(1)/len(y_train)
    print(y_prob)
    accuracy = testNaiveBayes(x_test,y_test,yCPT,nCPT,y_prob)
    
    return accuracy
    pass

In [98]:
X_train,X_test,Y_train,Y_test = train_test_split(X.to_numpy(),Y.to_numpy(),test_size=0.33)

In [99]:
accuracy = naiveBayesClassifier(X_train,Y_train,X_test,Y_test)

{'clump_thickness': {1: 0.019230769230769232, 2: 0.019230769230769232, 3: 0.038461538461538464, 4: 0.05128205128205128, 5: 0.1794871794871795, 6: 0.057692307692307696, 7: 0.12179487179487179, 8: 0.16025641025641027, 9: 0.05128205128205128, 10: 0.30128205128205127, '3': 0.0, '6': 0.0, '4': 0.0, '8': 0.0, '1': 0.0, '10': 0.0, '7': 0.0, '9': 0.0, '2': 0.0, '5': 0.0}, 'marginal_adhesion': {1: 0.14743589743589744, 2: 0.0641025641025641, 3: 0.11538461538461539, 4: 0.1282051282051282, 5: 0.0641025641025641, 6: 0.07051282051282051, 7: 0.04487179487179487, 8: 0.10897435897435898, 9: 0.019230769230769232, 10: 0.23717948717948717, '3': 0.0, '6': 0.0, '4': 0.0, '8': 0.0, '1': 0.0, '10': 0.0, '7': 0.0, '9': 0.0, '2': 0.0, '5': 0.0}, 'epithelial_size': {1: 0.00641025641025641, 2: 0.08974358974358974, 3: 0.14102564102564102, 4: 0.1987179487179487, 5: 0.14743589743589744, 6: 0.16025641025641027, 7: 0.03205128205128205, 8: 0.08333333333333333, 9: 0.00641025641025641, 10: 0.1346153846153846, '3': 0.0, '

In [100]:
accuracy

0.961038961038961